**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2021-2022/master/Assignment4/course_materials"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 559 kB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=N551c330cc4be413c8722aa2b54bb1ccc (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
print("RDFLib:")
#Subclass
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

print("SPARQL:")
q1 = prepareQuery(''' SELECT ?sub WHERE { 
    ?sub rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

for r in g.query(q1):
  print(r)

RDFLib:
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
SPARQL:
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [15]:
from rdflib.plugins.sparql import prepareQuery
print("RDFLib:")
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
#Subclass
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2,p2,o2 in g.triples((None, RDF.type, s)):
    print(s2)

print("SPARQL:")
q2= prepareQuery('''SELECT ?sub ?subclass WHERE { 
    { 
      ?sub rdf:type ns:Person. 
    } 
    UNION {
      # ?subclass rdfs:subClassOf* ns:Person.
      ?sub rdf:type/rdfs:subClassOf* ns:Person.
      ?sub rdf:type ?subclass
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)
for r in g.query(q2):
  print(r)

RDFLib:
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
SPARQL:
(rdflib.term.URIRef('http://somewhere#SaraJones'), None)
(rdflib.term.URIRef('http://somewhere#JohnSmith'), None)
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.URIRef('http://somewhere#Researcher'))


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [14]:

print("RDFLib\n")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for s2,p2,o2 in g.triples((s, None, None)):
      print(s2,p2,o2) 
#Subclass
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
 for s2,p2,o2 in g.triples((None, RDF.type, s)):
   for s3,p3,o3 in g.triples((s2, None, None)):
      print(s3,p3,o3)

print("SPARQL\n")
q3= prepareQuery('''
  SELECT ?sub ?obj ?pred WHERE { 
    { 
      ?sub rdf:type ns:Person.
      ?sub ?obj ?pred
    } UNION {
      # ?sub rdf:type ?x.
      # ?x rdfs:subClassOf* ns:Person.
          ?sub rdf:type/rdfs:subClassOf* ns:Person.
      ?sub ?obj ?pred.
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for r in g.query(q3):
  print(r)

RDFLib

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://som